In [2]:
import numpy as np
import pandas as pd

In [3]:
X = np.array([[0, 0],
              [0, 1], 
              [1, 0],
              [1, 1]])

y = np.array([[0],
              [1],
              [1], 
              [0]])

In [ ]:
class Layer:
    def __init__(self, input_size: int, neuron_count: int):
        self.W = np.random.randn(input_size, neuron_count) * 0.1
        self.b = np.zeros((1, neuron_count))
        
    def forward(self, X):
        return X @ self.W + self.b